In [6]:
!pip install opencv-python


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
import cv2
print(cv2.__version__)

sns.set(style="whitegrid")



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


ModuleNotFoundError: No module named 'cv2'

In [ ]:
train_csv_path = 'data/train_labels.csv'
df = pd.read_csv(train_csv_path)
df.head()

In [ ]:
df.info()


## Distribution of the target variable

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['bags_used'], kde=True, bins=20)
plt.title('Distribution of Bags Used to Fill Potholes')
plt.xlabel('Number of Bags')
plt.ylabel('Frequency')
plt.show()
df['bags_used'].describe()


## Reading the YOLO Annotations

In [ ]:
def read_yolo_annotation(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    annotations = []
    for line in lines:
        parts = line.strip().split()
        label = int(parts[0])
        x_center, y_center, width, height = map(float, parts[1:])
        annotations.append((label, x_center, y_center, width, height))
    return annotations
    

In [ ]:
annotation_files = glob('data/train_annotations/*.txt')


In [ ]:
sample_image_path = 'data/train_images/plabel.jpg'
sample_annotation_path = 'data/train_annotations/plabel.txt'

In [ ]:
image = cv2.imread(sample_image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
plt.title('Sample Image')
plt.show()

In [ ]:
annotations = read_yolo_annotation(sample_annotation_path)

# Convert YOLO format to bounding box coordinates
image_height, image_width, _ = image.shape
for annotation in annotations:
    label, x_center, y_center, width, height = annotation
    x_min = int((x_center - width / 2) * image_width)
    x_max = int((x_center + width / 2) * image_width)
    y_min = int((y_center - height / 2) * image_height)
    y_max = int((y_center + height / 2) * image_height)
    cv2.rectangle(image_rgb, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
    cv2.putText(image_rgb, str(label), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

# Display the image with annotations
plt.figure(figsize=(8, 8))
plt.imshow(image_rgb)
plt.title('Sample Image with Annotations')
plt.show()

In [ ]:
sample_images = glob('data/train_images/*.jpg')[:5]

plt.figure(figsize=(16, 16))
for i, image_path in enumerate(sample_images):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 5, i+1)
    plt.imshow(image_rgb)
    plt.axis('off')
plt.suptitle('Sample Images', fontsize=20)
plt.show()

In [ ]:
print("Summary of Target Variable - Bags Used")
print(df['bags_used'].describe())

print("\nConclusion from EDA:")
print("- The target variable 'bags_used' is right-skewed, with most potholes requiring a small number of bags.")
print("- Images have been successfully annotated, and bounding boxes appear to correctly identify potholes and measurement sticks.")
